# 데이터 분석 코드

# 응급실 데이터

In [3]:
import pandas as pd
import numpy as np
import warnings
from haversine import haversine
warnings.filterwarnings('ignore')

# 부산광역시 
# 응급실 실시간 데이터
pd.set_option('display.max_columns', None)
url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire?serviceKey=Zv%2FutM8zPV4Cpby0tQi%2FtiDmHPhc19Zr67HNB4XUKyLt89VHIxcmVtAdRSWES5ve3JQGe4P3OL6dyL6vChTt8Q%3D%3D&STAGE1=%EB%B6%80%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C&pageNo=1&numOfRows=100'
df1 = pd.read_xml(url, xpath='.//item')
df1_c = df1.copy()
df2 = df1_c[['dutyName','hpid','hvec','hvoc', 'dutyTel3']]
df2_c = df2.rename(columns={'dutyName':'병원명', 'hpid':'병원코드', 'hvoc':'수술실수', 'dutyTel3':'응급실연락처','hvec':'응급실수'})
# df2_c

# # 중증질환자 수용가능정보 실시간 데이터
url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire?serviceKey=Zv%2FutM8zPV4Cpby0tQi%2FtiDmHPhc19Zr67HNB4XUKyLt89VHIxcmVtAdRSWES5ve3JQGe4P3OL6dyL6vChTt8Q%3D%3D&STAGE1=%EB%B6%80%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C&pageNo=1&numOfRows=100'
df3 = pd.read_xml(url, xpath='.//item')
df3.columns = ['병원명','병원코드','뇌출혈','신생아','중증화상','뇌경색','응급실','심근경색','복부손상','사지접합','응급내시경','응급투석', '조산산모','정신질환자']
df3.drop(['병원명','정신질환자','응급실'],axis=1, inplace=True)
# df3

# # 병원 위치 데이터
url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire?serviceKey=Zv%2FutM8zPV4Cpby0tQi%2FtiDmHPhc19Zr67HNB4XUKyLt89VHIxcmVtAdRSWES5ve3JQGe4P3OL6dyL6vChTt8Q%3D%3D&Q0=%EB%B6%80%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C&pageNo=1&numOfRows=100'
df4 = pd.read_xml(url, xpath='.//item')
df5 = df4[['phpid','dutyAddr','wgs84Lat','wgs84Lon']]
df5_c = df5.rename(columns={'phpid':'병원코드','dutyAddr':'병원주소','wgs84Lat':'위도','wgs84Lon':'경도'})
# df5_c

# # 총 데이터 merge
dfx = pd.merge(df2_c, df5_c, on='병원코드')
dfa = pd.merge(dfx, df3, on='병원코드', how='outer')
# # dfa

# # 병원별 전체병상수, 응급실 포화도/상태 추가
er = pd.read_excel('er1.xlsx')
er.drop(['Unnamed: 0','병원명'], axis=1, inplace=True)
df_er = pd.merge(dfa, er, on='병원코드')
df_er['응급실포화도'] = (1-df_er['응급실수']/df_er['전체병상수'])*100
df_er['응급실포화상태'] = df_er['응급실포화도']
df_er['응급실포화상태'] = pd.cut(df_er['응급실포화상태'] ,bins=[0,30,60,90,np.inf],labels=["원활","보통","혼잡","매우혼잡"],right=False) #라벨 설정
df_er.insert(6, column= "거리", value= 0)

# 부산역을 현위치(A)로 설정
A = (35.11492,  129.04251)
for i in range(df_er.shape[0]):
    B = (df_er["위도"][i], df_er["경도"][i])
    df_er["거리"][i] = round(haversine(A, B), 3)

df_er

# csv로 저장
df_er.to_csv('hospital.csv', encoding='cp949', index=False)

# 질환 예측

In [19]:
# 질환별 증상데이터 + 전처리
d1 = pd.read_excel('fd_2.xlsx')
d1.drop('Unnamed: 0', axis=1, inplace=True)

# 이름이 같은 증상 제거
d1['협심증'] = d1['협심증'] + d1['협심증.1']
d1['호흡음 감소'] = d1['호흡음 감소.1'] + d1['호흡음 감소']
d1['흉부 압박'] = d1['흉부 압박.1'] + d1['흉부 압박']
d1['운동 시 호흡곤란'] = d1['운동 시 호흡곤란.1'] + d1['운동 시 호흡곤란']
d1['안면마비'] = d1['안면마비.1'] + d1['안면마비']
d1['피로'] = d1['피로.1'] + d1['피로']
d1['일반적인 불편함'] = d1['일반적인 불편함.1'] + d1['일반적인 불편함']
d1['경정맥 팽창'] = d1['경정맥 팽창.1'] + d1['경정맥 팽창']
d1['야간 땀'] = d1['야간 땀.1'] + d1['야간 땀']
d1['흉막 통증'] = d1['흉막 통증.1'] + d1['흉막 통증']
d1['비생산적인 기침'] = d1['비생산적인 기침.1'] + d1['비생산적인 기침']
d1['손의 마비'] = d1['손의 마비.1'] + d1['손의 마비']
d1['통증 가슴'] = d1['통증 가슴.1'] + d1['통증 가슴']
d1['압력 가슴'] = d1['압력 가슴'] + d1['압력 가슴.1']
d1['생산적인 기침'] = d1['생산적인 기침.1'] + d1['생산적인 기침']
d1['호흡 곤란'] = d1['호흡 곤란.1'] + d1['호흡 곤란']
d1['말이 어눌해졌다'] = d1['말이 어눌해졌다'] + d1['말이 어눌해졌다.1']
d1['발한 증가'] = d1['발한 증가.1'] + d1['발한 증가.1']
d1['노란 가래'] = d1['노란 가래.1'] + d1['노란 가래']
colsx = ['협심증.1', '호흡음 감소.1', '흉부 압박.1', '운동 시 호흡곤란.1', '안면마비.1', '피로.1', '일반적인 불편함.1', '경정맥 팽창.1', '현기증.1', '야간 땀.1', '비생산적인 기침.1', '손의 마비.1', '통증 가슴.1', '흉막 통증.1', '압력 가슴.1', '생산적인 기침.1', '호흡 곤란.1', '말이 어눌해졌다.1', '발한 증가.1', '노란 가래.1']
d1.drop(colsx, axis=1, inplace=True)

# 질환에 포함되지 않는 증상들 제거
col = d1.columns
a = []
for x in col:
    if d1[[x]].value_counts()[0]==24:
        a.append(x)
        
d2 = d1.drop(a, axis=1)
d2

d2.to_excel('disease.xlsx', index=False, encoding='utf-8')

# 모델링 (프로토타입)

In [21]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

x = d2.drop(['질병','수술'], axis=1)
y = d2['수술']

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.01,random_state=1234)

model1 = DecisionTreeClassifier()
model1.fit(x_train, y_train)
pred1 = model1.predict(x_test)
print(accuracy_score(pred1,y_test))

# print(x_test)
# print(pred1)

1.0


In [22]:
model2 = DecisionTreeClassifier()
model2.fit(x,y)
pred2 = model2.predict(x)
model2.score(x,y)

1.0

In [23]:
# 증상으로 예측 예시
patient = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
patient = np.array(patient)
patient = patient.reshape(1, 143)
pred3 = model2.predict(patient)[0]
print("="*17)
print("예측질병:", pred3)
print("="*17)
for i in df_er.columns:
    if pred3 == i:
        hospital = df_er.loc[df_er[i] == "Y", :]
hospital = hospital[(hospital['응급실수']>0)&(hospital['수술실수']>0)]
hospital = hospital.reset_index(drop=True)
hospital

예측질병: 심근경색


,병원명,병원코드,응급실수,수술실수,응급실연락처,병원주소,거리,위도,경도,뇌출혈,신생아,중증화상,뇌경색,심근경색,복부손상,사지접합,응급내시경,응급투석,조산산모,전체병상수,응급실포화도,응급실포화상태
0,부산성모병원(재단법인천주교부산교구유지재단),A1200013,8,7,051-933-7119,부산광역시 남구 용호로232번길 25-14 (용호동),6.086,35.110457,129.109192,Y,Y,N,Y,Y,Y,N,Y,Y,Y,19,57.894737,보통
1,의료법인정화의료재단김원묵기념봉생병원,A1200017,5,8,051-664-4119,"부산광역시 동구 중앙대로 401 (좌천동, 봉생병원)",1.918,35.130858,129.050571,Y,N,N,Y,Y,N,N,Y,Y,N,11,54.545455,보통
2,동래봉생병원,A1200011,8,5,051-520-5600,부산광역시 동래구 안연로109번길 27 (안락동),10.343,35.196946,129.096167,Y,N,N,Y,Y,N,N,N,N,N,10,20.000000,원활
3,대동병원,A1200018,11,6,051-550-9391,부산광역시 동래구 충렬대로 187 (명륜동),10.510,35.204273,129.080215,Y,N,N,N,Y,Y,N,N,N,N,21,47.619048,보통
4,의료법인온그룹의료재단온종합병원,A1200020,18,9,051-607-0139,"부산광역시 부산진구 가야대로 721, 719, 767 (당감동)",4.825,35.157877,129.049984,Y,N,N,Y,Y,Y,N,Y,Y,N,30,40.000000,보통
5,인제대학교부산백병원,A1200001,17,17,051-890-5995,"부산광역시 부산진구 복지로 75, 진사로83번길 81, 1층(일부), 3층 (개금동)",4.034,35.146454,129.020572,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,35,51.428571,보통
6,학교법인)동의병원,A1200019,8,1,051-850-8777,"부산광역시 부산진구 양정로 62, 지상2ㆍ3ㆍ10층 각 일부/ 지하1ㆍ지상1ㆍ지상4...",6.863,35.169922,129.076744,Y,N,N,Y,Y,N,N,Y,Y,N,17,52.941176,보통
7,구포성심병원,A1200045,5,4,051-330-2090,부산광역시 북구 낙동대로 1786 (구포동),11.085,35.209710,129.004751,Y,N,N,N,Y,Y,N,Y,Y,N,10,50.000000,보통
8,의료법인은성의료재단좋은삼선병원,A1200005,5,6,051-310-9416,부산광역시 사상구 가야대로 326 (주례동),5.045,35.150568,129.008198,Y,N,N,Y,Y,Y,Y,Y,Y,N,20,75.000000,혼잡
9,부산대학교병원,A1200002,9,26,051-240-7776,부산광역시 서구 구덕로 179 (아미동1가),2.620,35.101054,129.019222,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,32,71.875000,혼잡


In [21]:
print("="*17)
print("예측질병:", pred3)
print("="*17)
hospital["실시간응급실가용병상수"] = hospital["응급실수"]
hospital["실시간수술실가용병상수"] = hospital["수술실수"]
hospital["심근경색수술가능여부"] = hospital["심근경색"]

hospital[["병원명", "실시간응급실가용병상수", "실시간수술실가용병상수", "심근경색수술가능여부", "응급실포화상태"]]

예측질병: 심근경색


,병원명,실시간응급실가용병상수,실시간수술실가용병상수,심근경색수술가능여부,응급실포화상태
0,부산성모병원(재단법인천주교부산교구유지재단),7,7,Y,혼잡
1,의료법인정화의료재단김원묵기념봉생병원,6,7,Y,보통
2,동래봉생병원,6,5,Y,보통
3,대동병원,8,6,Y,혼잡
4,의료법인온그룹의료재단온종합병원,17,9,Y,보통
5,인제대학교부산백병원,2,15,Y,매우혼잡
6,학교법인)동의병원,4,1,Y,혼잡
7,구포성심병원,8,4,Y,원활
8,의료법인은성의료재단좋은삼선병원,7,7,Y,혼잡
9,의료법인은성의료재단좋은강안병원,12,9,Y,원활


In [4]:
# 프로토타입용 데이터 확인
oo = pd.read_csv("self.csv")
oo

,수술명,붓기,호흡곤란,기력없음,어지럼증,가슴답답함,저혈압,두통,의식저하,구토,고혈압,마비,감각이상,구역질,식은땀,발열,흉통,복통,이물질,열상,저체온,맥박이상,쇼크
0,응급투석,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,뇌출혈,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0
2,뇌경색,0,1,0,1,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0
3,심근경색,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0
4,응급내시경,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
5,중증화상,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
